# The Data Sammler
### This notebook will be used to gather city data and format it so that it can be exported to the data-set used for our MVP.

- We start out with the socio-economic data from https://www.numbeo.com/quality-of-life/in/Berlin as this is the hardest data to come by.
- We then add to the city data by scraping https://climatedata.eu/ or by manual input, depending on what sources we find.
- Lastly, we need to add the latitude and longitude for each city.
- The goal is to be able to update our city data set using this notebook only.

In [1]:
import pandas as pd
import lxml

## 1. Getting socio-economic data
### 1.1 We'll start by importing our sample data, to keep track of what we need new city data to look like.

In [2]:
city_data = pd.read_json("../data/Combined_data.json")
city_data.head()

,city,autumn_high,autumn_prec_days,autumn_sun_hrs,spring_high,spring_prec_days,spring_sun_hrs,summer_high,summer_prec_days,summer_sun_hrs,...,cost_of_living,health_care,pollution,property_income_ratio,purchasing_power,safety,traffic_time,quality_of_life,lat,lng
0,Amsterdam,14,17,98,13,15,166,21,13,203,...,84.18,69.45,30.79,10.98,81.63,67.32,29.88,168.38,52.350000,4.916667
1,Athens,23,8,212,20,9,235,31,2,347,...,59.28,56.17,57.30,12.75,40.69,50.49,37.98,119.84,37.983333,23.733333
2,Belgrade,18,7,153,18,9,182,26,8,266,...,40.49,53.69,63.57,22.22,34.87,62.02,35.89,107.89,44.833333,20.500000
3,Berlin,13,9,109,13,9,167,23,10,219,...,67.41,69.68,39.45,9.63,98.54,58.92,34.06,164.83,52.516667,13.400000
4,Bratislava,15,6,148,15,7,214,25,7,292,...,50.81,57.17,41.12,13.37,61.82,68.68,30.89,147.54,48.150000,17.116667


In [3]:
city_data.columns

Index(['city', 'autumn_high', 'autumn_prec_days', 'autumn_sun_hrs',
       'spring_high', 'spring_prec_days', 'spring_sun_hrs', 'summer_high',
       'summer_prec_days', 'summer_sun_hrs', 'winter_high', 'winter_prec_days',
       'winter_sun_hrs', 'climate', 'cost_of_living', 'health_care',
       'pollution', 'property_income_ratio', 'purchasing_power', 'safety',
       'traffic_time', 'quality_of_life', 'lat', 'lng'],
      dtype='object')

In [4]:
city_list = list(city_data.city)
print(city_list)

['Amsterdam', 'Athens', 'Belgrade', 'Berlin', 'Bratislava', 'Brussels', 'Bucharest', 'Budapest', 'Copenhagen', 'Dublin', 'Helsinki', 'Lisbon', 'Ljubljana', 'London', 'Luxembourg', 'Madrid', 'Nicosia', 'Oslo', 'Paris', 'Reykjavik', 'Riga', 'Rome', 'Sarajevo', 'Skopje', 'Sofia', 'Stockholm', 'Tallinn', 'Tirana', 'Valletta', 'Vienna', 'Vilnius', 'Warsaw', 'Zagreb']


### 1.2 Using 'Barcelona' as a test city, because it has information in Numbeo and Climatedata, we'll start by gathering socio-economic data.

What we want is to input batches of cities as lists and output a new dataframe that can be merged with our dataset. Any cities that end up in the error list will not be worked with at this time. In this case 'Bern', it has information in Numbeo, but not in a way that can be scraped by our function. We will add the information for it manually.

In [5]:
test = ['Barcelona', 'Bern']

In [6]:
def get_city_info(city_list) -> list:
    
    # create empty database
    df = pd.DataFrame()
    errors = []
    
    # iterate over list of cities to obtain table data from Numbeo
    for city in city_list:
        try:
            table = pd.read_html(f"https://www.numbeo.com/quality-of-life/in/{city}")[3]
            table = (table.assign(city=city)
                      .drop(columns=[2], axis=1)
                      .rename(columns={0:"category", 1:"numeral"})
                      .drop([table.index[8]]))
            df = df.append(table)
        
        except:
            errors.append(city)
    
    df.reset_index(inplace=True, drop=True)

    return df, errors

In [7]:
get_city_info(test)

(                         category  numeral       city
 0          Purchasing Power Index    61.85  Barcelona
 1                    Safety Index    55.41  Barcelona
 2               Health Care Index    77.55  Barcelona
 3                   Climate Index    95.73  Barcelona
 4            Cost of Living Index    64.35  Barcelona
 5  Property Price to Income Ratio    13.20  Barcelona
 6      Traffic Commute Time Index    30.07  Barcelona
 7                 Pollution Index    65.05  Barcelona
 8        ƒ Quality of Life Index:   137.34  Barcelona,
 ['Bern'])

In [8]:
test_df = get_city_info(test)

Then we'll pivot the table and rename the columns.

In [9]:
def pivot_cities(df):

    df = df.pivot(index="city", columns="category", values=["numeral"])
    
    colnames = ['climate', 'cost_of_living', 'health_care', 'pollution', 'property_income_ratio', 'purchasing_power',
                'safety', 'traffic_time', 'quality_of_life']
    df.columns = colnames
    df = df.reset_index()
    
    return df

In [10]:
test_pivot = pivot_cities(test_df[0])
test_pivot

,city,climate,cost_of_living,health_care,pollution,property_income_ratio,purchasing_power,safety,traffic_time,quality_of_life
0,Barcelona,95.73,64.35,77.55,65.05,13.2,61.85,55.41,30.07,137.34


In [11]:
test_pivot.dtypes

city                      object
climate                  float64
cost_of_living           float64
health_care              float64
pollution                float64
property_income_ratio    float64
purchasing_power         float64
safety                   float64
traffic_time             float64
quality_of_life          float64
dtype: object

In [12]:
# Final function for city data from Numbeo

def socio_ec(city_list):
    df = get_city_info(city_list)
    df = pivot_cities(df[0])
    
    return df
    

In [13]:
socio_ec(test)

,city,climate,cost_of_living,health_care,pollution,property_income_ratio,purchasing_power,safety,traffic_time,quality_of_life
0,Barcelona,95.73,64.35,77.55,65.05,13.2,61.85,55.41,30.07,137.34


## 2. Getting weather data
### 2.1 We can now take the new dataframe and add weather data. We need to get the url code for each city from climatedata, for the moment, we are doing this manually.

In [14]:
city_codes = {'Barcelona':'spxx0015'}

In [15]:
def get_weather(city_dict) -> dict:
    df = pd.DataFrame()
    errors = []

    for city, code in city_dict.items():
        try:
            # get first table (from jan to june)
            table1 = pd.read_html(f'https://www.climatedata.eu/climate.php?loc={code}&lang=en')[2]
        
            # get second table (from july to dec)
            table2 = pd.read_html(f'https://www.climatedata.eu/climate.php?loc={code}&lang=en')[3]
            
            # merge table1 and table2
            table_combo = table1.merge(table2)
            table_combo = table_combo.assign(city=city)
            
            # assign tables to df
            df = df.append(table_combo)
        
        except:
            errors.append(city)

    return df, errors

In [16]:
test_weather = get_weather(city_codes)[0]
test_weather

,Unnamed: 0,Jan,Feb,Mar,Apr,May,June,July,Aug,Sep,Oct,Nov,Dec,city
0,Average high in °C,13,15,16,18,21,24,28,28,26,22,17,14,Barcelona
1,Average low in °C,4,5,7,9,12,16,19,19,17,13,8,6,Barcelona
2,Av. precipitation - mm,41,29,42,49,59,42,20,61,85,91,58,51,Barcelona
3,Days with precip.,5,4,5,5,5,4,2,4,5,6,5,5,Barcelona
4,Hours of sunshine,149,163,200,220,244,262,310,282,219,180,146,138,Barcelona


### 2.2 We also want to provide the data as seaonal weather and get rid of '_prec_mm' and '_low'.

In [17]:
def get_season_avg(df):
    season_dict = {"winter": ["Dec", "Jan", "Feb"],
                   "spring": ["Mar", "Apr", "May"],
                   "summer": ["June", "July", "Aug"],
                   "autumn": ["Sep", "Oct", "Nov"],
                   }
    
    for season, month in season_dict.items():
        # Get season averages
        df[season] = round((df[month[0]] + df[month[1]] + df[month[2]]) / 3)
        df = (df.drop(df[month], axis=1)
                .rename(columns={"Unnamed: 0":"weather"}))
        
        df = df[df['weather'].isin(['Average high in °C', 'Days with precip.', 'Hours of sunshine'])]

    return df

In [18]:
test_season = get_season_avg(test_weather)

test_season

,weather,city,winter,spring,summer,autumn
0,Average high in °C,Barcelona,14.0,18.0,27.0,22.0
3,Days with precip.,Barcelona,5.0,5.0,3.0,5.0
4,Hours of sunshine,Barcelona,150.0,221.0,285.0,182.0


In [19]:
def transform_df(df):
    df = df.rename(columns={"Unnamed: 0":"weather"})
    colnames = ["autumn_high","autumn_prec_days", "autumn_sun_hrs",
                "spring_high", "spring_prec_days", "spring_sun_hrs",
                "summer_high", "summer_prec_days", "summer_sun_hrs",
                "winter_high", "winter_prec_days", "winter_sun_hrs",
               ]
    
    df = df.pivot_table(index="city", columns=["weather"])
    df.columns = df.columns.get_level_values(1)
    df.columns = [colnames]
    df.reset_index(inplace=True)
    
    return df

In [20]:
transform_df(test_season)

,city,autumn_high,autumn_prec_days,autumn_sun_hrs,spring_high,spring_prec_days,spring_sun_hrs,summer_high,summer_prec_days,summer_sun_hrs,winter_high,winter_prec_days,winter_sun_hrs
0,Barcelona,22.0,5.0,182.0,18.0,5.0,221.0,27.0,3.0,285.0,14.0,5.0,150.0


In [32]:
def season_weather(city_dict):
    df_tup = get_weather(city_dict)
    df = get_season_avg(df_tup[0])
    df = transform_df(df)
    df.columns = df.columns.get_level_values(0)
    
    return df

In [33]:
season_weather(city_codes)

,city,autumn_high,autumn_prec_days,autumn_sun_hrs,spring_high,spring_prec_days,spring_sun_hrs,summer_high,summer_prec_days,summer_sun_hrs,winter_high,winter_prec_days,winter_sun_hrs
0,Barcelona,22.0,5.0,182.0,18.0,5.0,221.0,27.0,3.0,285.0,14.0,5.0,150.0


## 3. Latitudes and Longitudes
### 3.1 The final piece of data is the lat and lng co-ordinates that we need to plot cities on a map.

We will be using 2 data sets, one for capital cities and one for all others. As both sets are not complete lists, we need both.

In [60]:
# first we need to import the data
lat_lng = pd.read_csv('../data/country-capitals.csv', error_bad_lines=False)
lat_lng.head()

b'Skipping line 229: expected 6 fields, saw 7\nSkipping line 240: expected 6 fields, saw 7\n'


,CountryName,CapitalName,CapitalLatitude,CapitalLongitude,CountryCode,ContinentName
0,Somaliland,Hargeisa,9.550000,44.050000,NaN,Africa
1,South Georgia and South Sandwich Islands,King Edward Point,-54.283333,-36.500000,GS,Antarctica
2,French Southern and Antarctic Lands,Port-aux-Français,-49.350000,70.216667,TF,Antarctica
3,Palestine,Jerusalem,31.766667,35.233333,PS,Asia
4,Aland Islands,Mariehamn,60.116667,19.900000,AX,Europe


In [70]:
lat_lng.loc[lat_lng["CapitalName"] == 'Prague']

,CountryName,CapitalName,CapitalLatitude,CapitalLongitude,CountryCode,ContinentName
65,Czech Republic,Prague,50.083333,14.466667,CZ,Europe


In [62]:
city_ort = (lat_lng[lat_lng["CapitalName"].isin(test)]
                                          .reset_index(drop=True)
                                          .rename(columns={"CapitalName":"city",
                                                           "CapitalLatitude":"lat",
                                                           "CapitalLongitude":"lng"}))
city_ort = city_ort[["city", "lat", "lng"]]

city_ort.head()

,city,lat,lng
0,Bern,46.916667,7.466667


## 4. Combining socio-ec, weather data and latitude + longitude
### 4.1 First, we want a function that combines the 2 new dataframes we created and the we want to add it to one all-encompasing function that takes the city name and code (for weather data) and adds it to our city_dataset.

In [34]:
soc_df = socio_ec(test)

In [35]:
soc_df

,city,climate,cost_of_living,health_care,pollution,property_income_ratio,purchasing_power,safety,traffic_time,quality_of_life
0,Barcelona,95.73,64.35,77.55,65.05,13.2,61.85,55.41,30.07,137.34


In [36]:
weather_df = season_weather(city_codes)

In [38]:
weather_df

,city,autumn_high,autumn_prec_days,autumn_sun_hrs,spring_high,spring_prec_days,spring_sun_hrs,summer_high,summer_prec_days,summer_sun_hrs,winter_high,winter_prec_days,winter_sun_hrs
0,Barcelona,22.0,5.0,182.0,18.0,5.0,221.0,27.0,3.0,285.0,14.0,5.0,150.0


In [40]:
soc_df.merge(weather_df)

,city,climate,cost_of_living,health_care,pollution,property_income_ratio,purchasing_power,safety,traffic_time,quality_of_life,...,autumn_sun_hrs,spring_high,spring_prec_days,spring_sun_hrs,summer_high,summer_prec_days,summer_sun_hrs,winter_high,winter_prec_days,winter_sun_hrs
0,Barcelona,95.73,64.35,77.55,65.05,13.2,61.85,55.41,30.07,137.34,...,182.0,18.0,5.0,221.0,27.0,3.0,285.0,14.0,5.0,150.0


### 4.2 Because the socio-ec data only needs the city name, we can just pass the city code dict as an input for our function.

In [58]:
def data_prep(city_dict):
    city_list = []
    
    # First the socio-economic function takes a list
    for key in city_dict:
        city_list.append(key)
    
    df_sc = socio_ec(city_list)
    
    # The weather function works with the complete dict
    df_we = season_weather(city_dict)
    
    # Merging just takes the two dataframes
    df_combo = df_sc.merge(df_we)
    
    return df_combo

In [59]:
data_prep(city_codes)

,city,climate,cost_of_living,health_care,pollution,property_income_ratio,purchasing_power,safety,traffic_time,quality_of_life,...,autumn_sun_hrs,spring_high,spring_prec_days,spring_sun_hrs,summer_high,summer_prec_days,summer_sun_hrs,winter_high,winter_prec_days,winter_sun_hrs
0,Barcelona,95.73,64.35,77.55,65.05,13.2,61.85,55.41,30.07,137.34,...,182.0,18.0,5.0,221.0,27.0,3.0,285.0,14.0,5.0,150.0


## 5. Exporting
### 5.1 We now want to send our new data to our city_dataset. All export codes will be commented out (until needed), so that rerunning this notebook will not automatically update our dataset.

In [ ]:
# city_data = pd.read_json("../data/Combined_data.json")